<a href="https://colab.research.google.com/github/Rex-Chan99/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta
!pip install numpy==1.23.5
!pip install pandas

In [ ]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

import pandas as pd
import pandas_ta as ta
import csv
import numpy as np

btc_data = pd.read_csv('Bitcoin Historical Data.csv', on_bad_lines='skip', lineterminator='\n')
btc_data.tail(5)

Cloning into 'Bitcoin_prediction'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 61 (delta 16), reused 8 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 3.80 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction


,Date,Price,Open,High,Low,Vol. ('000),Change %\r
4107,1/5/2014,"1,014.70",924.7,"1,029.90",911.4,21.37,9.74%\r
4108,1/4/2014,924.7,884.3,932.2,848.3,14.24,4.57%\r
4109,1/3/2014,884.3,856.9,888.2,839.4,9.71,3.19%\r
4110,1/2/2014,856.9,815.9,886.2,810.5,12.81,5.02%\r
4111,1/1/2014,815.9,805.9,829.9,771,10.76,1.24%\r


In [ ]:
# Convert the dictionary to a DataFrame
btc_data = pd.DataFrame(btc_data)

# 1. Parse the "Date" column into datetime format
btc_data["Date"] = pd.to_datetime(btc_data["Date"], format="%m/%d/%Y")

# 2. Sort the data by date in ascending order
btc_data.sort_values(by="Date", ascending=True, inplace=True)

In [ ]:
# Calculate RSI, EMA, SMA, and MACD
btc_data["RSI"] = ta.rsi(btc_data["Price"], length=14)  # Relative Strength Index
btc_data["EMA"] = ta.ema(btc_data["Price"], length=14)  # Exponential Moving Average
btc_data["SMA"] = ta.sma(btc_data["Price"], length=14)  # Simple Moving Average

# Calculate MACD
macd = ta.macd(btc_data["Price"], fast=12, slow=26, signal=9)
btc_data["MACD"] = macd["MACD_12_26_9"]
btc_data["MACD_Signal"] = macd["MACDs_12_26_9"]
btc_data["MACD_Hist"] = macd["MACDh_12_26_9"]

# Categorical MACD - Buy (1) and Sell (0) signals
btc_data["C_MACD"] = None  # Default to None (no signal)
btc_data.loc[
    (btc_data["MACD"] > btc_data["MACD_Signal"]) &
    (btc_data["MACD"].shift(1) <= btc_data["MACD_Signal"].shift(1)),
    "C_MACD"] = 1  # Buy signal # Detect Buy Signal (Crossover Up)
btc_data.loc[
    (btc_data["MACD"] <= btc_data["MACD_Signal"]) &
    (btc_data["MACD"].shift(1) > btc_data["MACD_Signal"].shift(1)),
    "C_MACD"] = 0  # Sell signal # Detect Sell Signal (Crossover Down)

# Calculate Bollinger Bands
bbands = ta.bbands(btc_data["Price"], length=20, std=2)
btc_data["BB_Lower"] = bbands["BBL_20_2.0"]  # Lower Band
btc_data["BB_Upper"] = bbands["BBU_20_2.0"]  # Upper Band
btc_data["BB_Middle"] = bbands["BBM_20_2.0"]  # Middle Band

# Categorical C_BB - Generate Buy and Sell Signals
btc_data["C_BB"] = None  # Default to no signal
btc_data.loc[btc_data["Price"] <= btc_data["BB_Lower"], "C_BB"] = 1  # Buy signal (oversold)
btc_data.loc[btc_data["Price"] >= btc_data["BB_Upper"], "C_BB"] = 0  # Sell signal (overbought)


# Display the first few rows of the data with calculated indicators
btc_data.tail()

,Date,Price,Open,High,Low,Vol. ('000),Change %\r,RSI,EMA,SMA,MACD,MACD_Signal,MACD_Hist
4,2025-04-01,NaN,"82,548.60",NaN,NaN,66.72,3.17%\r,95.393727,964.420986,NaN,11.7338,11.7338,-7.105427e-15
3,2025-04-02,NaN,"85,166.00",NaN,NaN,104.05,-3.10%\r,95.393727,964.420986,NaN,11.7338,11.7338,-7.105427e-15
2,2025-04-03,NaN,"82,522.60",NaN,NaN,83.15,0.77%\r,95.393727,964.420986,NaN,11.7338,11.7338,-7.105427e-15
1,2025-04-04,NaN,"83,160.20",NaN,NaN,91.54,0.85%\r,95.393727,964.420986,NaN,11.7338,11.7338,-7.105427e-15
0,2025-04-05,NaN,"83,876.00",NaN,NaN,32.71,-0.38%\r,95.393727,964.420986,NaN,11.7338,11.7338,-7.105427e-15


In [ ]:
# Calculate Bollinger Bands
bollinger_bands = ta.bbands(btc_data["Price"], length=20, std=2)
btc_data["BB_Upper"] = bollinger_bands["BBU_20_2.0"]
btc_data["BB_Middle"] = bollinger_bands["BBM_20_2.0"]
btc_data["BB_Lower"] = bollinger_bands["BBL_20_2.0"]

# Percentage difference between Bollinger Bands
btc_data["BB_Percentage"] = (
    (btc_data["BB_Upper"] - btc_data["BB_Lower"]) / btc_data["BB_Middle"])

In [ ]:
# Function to clean and convert volume data
def clean_volume(volume):
    if isinstance(volume, str):  # Check if the value is a string
        volume = volume.replace(',', '')  # Remove commas
        if 'B' in volume:  # If the value contains 'B' (billions)
            return float(volume.replace('B', '')) * 1_000_000_000
        elif 'M' in volume:  # If the value contains 'M' (millions)
            return float(volume.replace('M', '')) * 1_000_000
        elif 'K' in volume:  # If the value contains 'K' (thousands)
            return float(volume.replace('K', '')) * 1_000
        else:  # If no suffix is present, convert to float directly
            return float(volume)
    return np.nan  # Handle unexpected cases

# Apply the cleaning function to the volume column
btc_data["Volume"] = btc_data["Vol. ('000)"].apply(clean_volume)

In [ ]:
# Add On-Balance Volume (OBV)
btc_data["OBV"] = ta.obv(btc_data["Price"], btc_data["Volume"])

In [ ]:
# Day of the week (0=Monday, 6=Sunday)
btc_data["Day_of_Week"] = btc_data["Date"].dt.dayofweek

# Week of the year
btc_data["Week_of_Year"] = btc_data["Date"].dt.isocalendar().week

# Month of the year
btc_data["Month"] = btc_data["Date"].dt.month

# Quarter of the year
btc_data["Quarter"] = btc_data["Date"].dt.quarter

# Year
btc_data["Year"] = btc_data["Date"].dt.year

In [ ]:
# Support and Resistance Levels
btc_data["Support"] = btc_data["Low"].rolling(window=20).min()  # Lowest low in the past 20 days
btc_data["Resistance"] = btc_data["High"].rolling(window=20).max()  # Highest high in the past 20 days

In [ ]:
# Calculate the percentage price change over the next 7 days (Target Variable for Regression)
btc_data["Pct_Change"] = ((btc_data["Price"].shift(-7) - btc_data["Price"]) / btc_data["Price"]) * 100

In [ ]:
# Drop the original "Vol. ('000)" column
btc_data.drop(columns=["Vol. ('000)"], inplace=True)

# Reset index
btc_data.reset_index(drop=True, inplace=True)

In [ ]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           4112 non-null   datetime64[ns]
 1   Price          1129 non-null   float64       
 2   Open           4112 non-null   object        
 3   High           1116 non-null   float64       
 4   Low            1144 non-null   float64       
      4112 non-null   object        
 6   RSI            4093 non-null   float64       
 7   EMA            4099 non-null   float64       
 8   SMA            1074 non-null   float64       
 9   MACD           4087 non-null   float64       
 10  MACD_Signal    4079 non-null   float64       
 11  MACD_Hist      4079 non-null   float64       
 12  BB_Upper       1061 non-null   float64       
 13  BB_Middle      1061 non-null   float64       
 14  BB_Lower       1061 non-null   float64       
 15  BB_Percentage  1061 non-null   floa

In [ ]:
btc_data.head()

,Date,Price,Open,High,Low,Change %\r,RSI,EMA,SMA,MACD,...,Volume,OBV,Day_of_Week,Week_of_Year,Month,Quarter,Year,Support,Resistance,Pct_Change
0,2014-01-01,815.9,805.9,829.9,771.0,1.24%\r,NaN,NaN,NaN,NaN,...,10.76,10.76,2,1,1,1,2014,NaN,NaN,15.063120
1,2014-01-02,856.9,815.9,886.2,810.5,5.02%\r,NaN,NaN,NaN,NaN,...,12.81,23.57,3,1,1,1,2014,NaN,NaN,9.347649
2,2014-01-03,884.3,856.9,888.2,839.4,3.19%\r,NaN,NaN,NaN,NaN,...,9.71,33.28,4,1,1,1,2014,NaN,NaN,8.311659
3,2014-01-04,924.7,884.3,932.2,848.3,4.57%\r,NaN,NaN,NaN,NaN,...,14.24,47.52,5,1,1,1,2014,NaN,NaN,NaN
4,2014-01-05,NaN,924.7,NaN,911.4,9.74%\r,NaN,NaN,NaN,NaN,...,21.37,NaN,6,1,1,1,2014,NaN,NaN,NaN


In [ ]:
btc_data.tail()

,Date,Price,Open,High,Low,Change %\r,RSI,EMA,SMA,MACD,...,Volume,OBV,Day_of_Week,Week_of_Year,Month,Quarter,Year,Support,Resistance,Pct_Change
4107,2025-04-01,NaN,"82,548.60",NaN,NaN,3.17%\r,95.393727,964.420986,NaN,11.7338,...,66.72,NaN,1,14,4,2,2025,NaN,NaN,NaN
4108,2025-04-02,NaN,"85,166.00",NaN,NaN,-3.10%\r,95.393727,964.420986,NaN,11.7338,...,104.05,NaN,2,14,4,2,2025,NaN,NaN,NaN
4109,2025-04-03,NaN,"82,522.60",NaN,NaN,0.77%\r,95.393727,964.420986,NaN,11.7338,...,83.15,NaN,3,14,4,2,2025,NaN,NaN,NaN
4110,2025-04-04,NaN,"83,160.20",NaN,NaN,0.85%\r,95.393727,964.420986,NaN,11.7338,...,91.54,NaN,4,14,4,2,2025,NaN,NaN,NaN
4111,2025-04-05,NaN,"83,876.00",NaN,NaN,-0.38%\r,95.393727,964.420986,NaN,11.7338,...,32.71,NaN,5,14,4,2,2025,NaN,NaN,NaN
